In [1]:
import pandas as pd
import math
import re

In [2]:
re_han = re.compile("([\u4E00-\u9FD5]+)", re.U)

def txtcnt(txt):
    cnt = 0
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            cnt += len(blk)
    return cnt

def tokens(txt,word_len):
    words = []
    blocks = re_han.split(txt)
    for blk in blocks:
        if not blk:
            continue
        if re_han.match(blk):
            words += ngram(blk,word_len)
    return words

def ngram(txt,word_len):
    words = []
    tmp_txt = txt
    remain_len = len(tmp_txt) - word_len
    if remain_len <= 0:
        words.append(tmp_txt)
    else:
        inx = 0
        while inx <= remain_len:
            word = tmp_txt[inx:inx+word_len]
            words.append(word)
            inx += 1
    return words

In [3]:
word_len = 6
line_inx = 1
file_name = 'comment_word_'+ str(word_len) +'.txt'
with open('comment_all.txt','r') as f \
    ,open(file_name,'w') as w:
    for line in f.readlines():
        words = tokens(line,word_len)
        for word in words:
            w.write(word + ',' + str(line_inx) +',1\n')
        line_inx += 1

In [4]:
df = pd.read_csv(file_name,header=None)
df[:2]

,0,1,2
0,难受,1,1
1,问题不在福祉,2,1


In [5]:
idf = df.filter(items=[0,1])

In [6]:
len(idf)

37971

In [7]:
idf_df = idf.drop_duplicates()

In [8]:
idf_df1 = idf_df.groupby(by=0).count()

In [9]:
idf_df1[:5]

,1
0,
一,3
一一从中读到,1
一一张罚单丢,1
一万这还不包,1
一下不就行了,1


In [10]:
tf = df.filter(items=[0,2])

In [11]:
tf_df = tf.groupby(by=0).sum()

In [12]:
tf_df[:5]

,2
0,
一,3
一一从中读到,1
一一张罚单丢,1
一万这还不包,1
一下不就行了,1


In [13]:
tf_df.sort_values(by=2,ascending=False).to_csv('词频.csv')

In [14]:
tf_idf = idf_df1.join(tf_df)

In [15]:
tf_idf['idf'] = tf_idf[1].apply(lambda x: math.log(337.0/x))
tf_idf['tf'] = tf_idf[2]/470.0

In [16]:
tf_idf['score'] = tf_idf['idf'] * tf_idf['tf']

In [17]:
tf_idf[:5]

,1,2,idf,tf,score
0,,,,,
一,3,3,4.721471,0.006383,0.030137
一一从中读到,1,1,5.820083,0.002128,0.012383
一一张罚单丢,1,1,5.820083,0.002128,0.012383
一万这还不包,1,1,5.820083,0.002128,0.012383
一下不就行了,1,1,5.820083,0.002128,0.012383


In [18]:
tf_idf1 = tf_idf.reset_index()

In [19]:
tf_idf1.sort_values(by='score',ascending=False)

,0,1,2,idf,tf,score
31751,转发微博,34,34,2.293722,0.072340,0.165929
11099,唉,30,30,2.418886,0.063830,0.154397
6941,元,14,16,3.181026,0.034043,0.108290
11014,哎,16,16,3.047494,0.034043,0.103744
10793,呵呵,15,15,3.112033,0.031915,0.099320
...,...,...,...,...,...,...
12073,基本利益得到,1,1,5.820083,0.002128,0.012383
12072,基本不花钱,1,1,5.820083,0.002128,0.012383
12071,基本上路的原,1,1,5.820083,0.002128,0.012383
12070,基本上就是百,1,1,5.820083,0.002128,0.012383


In [20]:
tf_idf1[(tf_idf1[2]>=3) & (tf_idf1[1]>=3)].sort_values(by='score',ascending=False)

,0,1,2,idf,tf,score
31751,转发微博,34,34,2.293722,0.072340,0.165929
11099,唉,30,30,2.418886,0.063830,0.154397
6941,元,14,16,3.181026,0.034043,0.108290
11014,哎,16,16,3.047494,0.034043,0.103744
10793,呵呵,15,15,3.112033,0.031915,0.099320
...,...,...,...,...,...,...
15815,开运输行业各,3,3,4.721471,0.006383,0.030137
16308,必将会计入成,3,3,4.721471,0.006383,0.030137
16384,快递都离不开,3,3,4.721471,0.006383,0.030137
141,一件快递都离,3,3,4.721471,0.006383,0.030137


In [21]:
tf_idf1.sort_values(by='score',ascending=False).to_csv('score.csv',index=False)

In [22]:
tf_idf1[:1]

,0,1,2,idf,tf,score
0,一,3,3,4.721471,0.006383,0.030137


In [23]:
import math

math.log(10.0)

2.302585092994046